In [1]:
import nfl_data_py as nfl
import numpy as np
import pandas as pd
import keras

In [2]:
season_data = nfl.import_seasonal_pfr('rec', [2018,2019,2020,2021])

In [3]:
season_data

,season,player,pfr_id,tm,age,pos,g,gs,tgt,rec,...,yac,yac_r,adot,brk_tkl,rec_br,drop,drop_percent,int,rat,loaded
0,2018,Michael Thomas,ThomMi05,NO,25.0,WR,16.0,16.0,147.0,125.0,...,514.0,4.1,7.7,6.0,20.8,3.0,0.020,1.0,124.1,2023-08-21
1,2018,Zach Ertz,ErtzZa00,PHI,28.0,TE,16.0,16.0,156.0,116.0,...,353.0,3.0,7.2,3.0,38.7,5.0,0.032,5.0,98.9,2023-08-21
2,2018,DeAndre Hopkins,HopkDe00,HOU,26.0,WR,16.0,16.0,163.0,115.0,...,387.0,3.4,11.5,5.0,23.0,2.0,0.012,3.0,115.9,2023-08-21
3,2018,Julio Jones,JoneJu02,ATL,29.0,WR,16.0,16.0,170.0,113.0,...,448.0,4.0,14.0,3.0,37.7,9.0,0.053,3.0,106.9,2023-08-21
4,2018,Adam Thielen,ThieAd00,MIN,28.0,WR,16.0,16.0,153.0,113.0,...,416.0,3.7,9.4,3.0,37.7,3.0,0.020,2.0,115.2,2023-08-21
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2081,2021,Vyncint Smith,SmitVy00,NYJ,25.0,WR,2.0,0.0,1.0,0.0,...,0.0,NaN,18.0,0.0,NaN,0.0,0.000,0.0,39.6,2023-08-21
2082,2021,Stephen Sullivan,SullSt00,CAR,25.0,TE,3.0,0.0,1.0,0.0,...,0.0,NaN,-2.0,0.0,NaN,1.0,1.000,0.0,39.6,2023-08-21
2083,2021,Colin Thompson,ThomCo03,CAR,28.0,TE,11.0,2.0,1.0,0.0,...,0.0,NaN,5.0,0.0,NaN,0.0,0.000,0.0,39.6,2023-08-21
2084,2021,Trent Williams,WillTr21,SF,33.0,LT,15.0,15.0,1.0,0.0,...,0.0,NaN,1.0,0.0,NaN,0.0,0.000,0.0,39.6,2023-08-21


In [4]:
raw_data = pd.DataFrame(season_data,
                         columns = ['season', 'player', 
                                    'tm', 'pos','tgt','rec','yds','td','yac_r','drop_percent','g'])

In [5]:
#raw_data

In [6]:
wr_data_train = raw_data[raw_data['pos'] =='WR']
wr_data_train = wr_data_train[wr_data_train['tgt'] >= 50]
wr_data_2021 = wr_data_train[wr_data_train['season'] == 2021]
wr_data_2020 = wr_data_train[wr_data_train['season'] == 2020]

In [7]:
#wr_data_2020

In [8]:
wr_data_2021 = wr_data_2021.drop(columns = ['season','tm'])
wr_data_2020 = wr_data_2020.drop(columns = ['season','tm'])

In [9]:
wr_data_2021

,player,pos,tgt,rec,yds,td,yac_r,drop_percent,g
1551,Cooper Kupp,WR,191.0,145.0,1947.0,16.0,5.8,0.042,17.0
1552,Davante Adams,WR,169.0,123.0,1553.0,11.0,4.8,0.018,16.0
1553,Tyreek Hill,WR,159.0,111.0,1239.0,9.0,4.0,0.044,17.0
1554,Justin Jefferson,WR,167.0,108.0,1616.0,10.0,4.5,0.042,17.0
1556,Diontae Johnson,WR,169.0,107.0,1161.0,8.0,4.9,0.030,16.0
...,...,...,...,...,...,...,...,...,...
1705,Calvin Ridley,WR,52.0,31.0,281.0,2.0,2.9,0.058,5.0
1708,Olamide Zaccheaus,WR,53.0,31.0,406.0,3.0,3.0,0.038,17.0
1719,Keelan Cole,WR,51.0,28.0,449.0,1.0,3.0,0.020,15.0
1732,Darius Slayton,WR,58.0,26.0,339.0,2.0,2.5,0.103,13.0


In [10]:
wr_data_2021 = wr_data_2021.reset_index()
wr_data_2020 = wr_data_2020.reset_index()

In [11]:
def calc_fppg(rec, yds, td, g):
    fppg = ((rec*1.0 + yds*.1 + td*6.0)/g)
    return fppg

In [12]:
wr_data_2021['fppg'] = np.empty(len(wr_data_2021), float).tolist()

In [13]:
for i in range(0,len(wr_data_2021)):
    wr_data_2021.loc[i,'fppg'] = calc_fppg(wr_data_2021.iloc[i]['rec'],
                                         wr_data_2021.iloc[i]['yds'],
                                         wr_data_2021.iloc[i]['td'], wr_data_2021.iloc[i]['g'])

In [14]:
wr_data_2021 = wr_data_2021.rename(columns={'tgt':'tgt/g_2021', 'rec':'rec/g_2021', 'yds':'yds/g_2021', 'td':'td/g_2021', 'yac_r':'yac_r_2021'
                   , 'drop_percent':'drop_percent_2021', 'g':'g_2021', 'fppg':'fppg_2021'})

In [15]:
wr_data_2021

,index,player,pos,tgt/g_2021,rec/g_2021,yds/g_2021,td/g_2021,yac_r_2021,drop_percent_2021,g_2021,fppg_2021
0,1551,Cooper Kupp,WR,191.0,145.0,1947.0,16.0,5.8,0.042,17.0,25.629412
1,1552,Davante Adams,WR,169.0,123.0,1553.0,11.0,4.8,0.018,16.0,21.518750
2,1553,Tyreek Hill,WR,159.0,111.0,1239.0,9.0,4.0,0.044,17.0,16.994118
3,1554,Justin Jefferson,WR,167.0,108.0,1616.0,10.0,4.5,0.042,17.0,19.388235
4,1556,Diontae Johnson,WR,169.0,107.0,1161.0,8.0,4.9,0.030,16.0,16.943750
...,...,...,...,...,...,...,...,...,...,...,...
88,1705,Calvin Ridley,WR,52.0,31.0,281.0,2.0,2.9,0.058,5.0,14.220000
89,1708,Olamide Zaccheaus,WR,53.0,31.0,406.0,3.0,3.0,0.038,17.0,5.270588
90,1719,Keelan Cole,WR,51.0,28.0,449.0,1.0,3.0,0.020,15.0,5.260000
91,1732,Darius Slayton,WR,58.0,26.0,339.0,2.0,2.5,0.103,13.0,5.530769


In [16]:
wr_data_2020['fppg'] = np.empty(len(wr_data_2020), float).tolist()

In [17]:
for i in range(0,len(wr_data_2020)):
    wr_data_2020.loc[i,'fppg'] = calc_fppg(wr_data_2020.iloc[i]['rec'],
                                         wr_data_2020.iloc[i]['yds'],
                                         wr_data_2020.iloc[i]['td'], wr_data_2020.iloc[i]['g'])

In [18]:
wr_data_2020 = wr_data_2020.rename(columns={'tgt':'tgt/g_2020', 'rec':'rec/g_2020', 'yds':'yds/g_2020', 'td':'td/g_2020', 'yac_r':'yac_r_2020'
                   , 'drop_percent':'drop_percent_2020', 'g':'g_2020', 'fppg':'fppg_2020'})

In [19]:
wr_data_2020

,index,player,pos,tgt/g_2020,rec/g_2020,yds/g_2020,td/g_2020,yac_r_2020,drop_percent_2020,g_2020,fppg_2020
0,1026,Stefon Diggs,WR,166.0,127.0,1535.0,8.0,3.7,0.048,16.0,20.531250
1,1027,Davante Adams,WR,149.0,115.0,1374.0,18.0,5.2,0.007,14.0,25.742857
2,1028,DeAndre Hopkins,WR,160.0,115.0,1407.0,6.0,4.6,0.013,16.0,18.231250
3,1031,Allen Robinson,WR,151.0,102.0,1250.0,6.0,3.1,0.020,16.0,16.437500
4,1032,Keenan Allen,WR,147.0,100.0,992.0,8.0,4.4,0.020,14.0,17.657143
...,...,...,...,...,...,...,...,...,...,...,...
82,1172,Jalen Reagor,WR,54.0,31.0,396.0,1.0,6.2,0.019,11.0,6.963636
83,1174,KJ Hamler,WR,56.0,30.0,381.0,3.0,4.1,0.125,13.0,6.623077
84,1176,Breshad Perriman,WR,60.0,30.0,505.0,3.0,5.3,0.050,12.0,8.208333
85,1178,James Washington,WR,56.0,30.0,392.0,5.0,4.6,0.071,16.0,6.200000


In [20]:
full_wr_data = pd.merge(wr_data_2020, wr_data_2021, on = 'player')

In [21]:
full_wr_data = full_wr_data.drop(columns = ['pos_x', 'pos_y', 'index_y', 'index_x',])

In [22]:
full_wr_data['yds/g_2020'] = full_wr_data['yds/g_2020']/full_wr_data['g_2020']
full_wr_data['tgt/g_2020'] = full_wr_data['tgt/g_2020']/full_wr_data['g_2020']
full_wr_data['rec/g_2020'] = full_wr_data['rec/g_2020']/full_wr_data['g_2020']
full_wr_data['td/g_2020'] = full_wr_data['td/g_2020']/full_wr_data['g_2020']
full_wr_data['yds/g_2021'] = full_wr_data['yds/g_2021']/full_wr_data['g_2021']
full_wr_data['tgt/g_2021'] = full_wr_data['tgt/g_2021']/full_wr_data['g_2021']
full_wr_data['rec/g_2021'] = full_wr_data['rec/g_2021']/full_wr_data['g_2021']
full_wr_data['td/g_2021'] = full_wr_data['td/g_2021']/full_wr_data['g_2021']

In [23]:
full_wr_data

,player,tgt/g_2020,rec/g_2020,yds/g_2020,td/g_2020,yac_r_2020,drop_percent_2020,g_2020,fppg_2020,tgt/g_2021,rec/g_2021,yds/g_2021,td/g_2021,yac_r_2021,drop_percent_2021,g_2021,fppg_2021
0,Stefon Diggs,10.375000,7.937500,95.937500,0.500000,3.7,0.048,16.0,20.531250,9.647059,6.058824,72.058824,0.588235,3.2,0.030,17.0,16.794118
1,Davante Adams,10.642857,8.214286,98.142857,1.285714,5.2,0.007,14.0,25.742857,10.562500,7.687500,97.062500,0.687500,4.8,0.018,16.0,21.518750
2,DeAndre Hopkins,10.000000,7.187500,87.937500,0.375000,4.6,0.013,16.0,18.231250,6.400000,4.200000,57.200000,0.800000,3.3,0.016,10.0,14.720000
3,Allen Robinson,9.437500,6.375000,78.125000,0.375000,3.1,0.020,16.0,16.437500,5.500000,3.166667,34.166667,0.083333,2.7,0.030,12.0,7.083333
4,Keenan Allen,10.500000,7.142857,70.857143,0.571429,4.4,0.020,14.0,17.657143,9.812500,6.625000,71.125000,0.375000,3.2,0.051,16.0,15.987500
5,Tyler Lockett,8.250000,6.250000,65.875000,0.625000,3.3,0.061,16.0,16.587500,6.687500,4.562500,73.437500,0.500000,3.8,0.000,16.0,14.906250
6,Chosen Anderson,8.500000,5.937500,68.500000,0.187500,5.2,0.044,16.0,13.912500,6.470588,3.117647,30.529412,0.294118,3.0,0.064,17.0,7.935294
7,Amari Cooper,8.125000,5.750000,69.625000,0.312500,4.5,0.023,16.0,14.587500,6.933333,4.533333,57.666667,0.533333,3.5,0.029,15.0,13.500000
8,Cooper Kupp,8.266667,6.133333,64.933333,0.200000,5.7,0.048,15.0,13.826667,11.235294,8.529412,114.529412,0.941176,5.8,0.042,17.0,25.629412
9,Calvin Ridley,9.533333,6.000000,91.600000,0.600000,3.1,0.042,15.0,18.760000,10.400000,6.200000,56.200000,0.400000,2.9,0.058,5.0,14.220000


In [24]:
from sklearn.model_selection import train_test_split
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error

In [25]:
y_yds = full_wr_data['yds/g_2021']

In [26]:
x_yds = full_wr_data.drop(columns = ['player', 'tgt/g_2021', 'rec/g_2021', 'yds/g_2021'
                                     , 'td/g_2021', 'yac_r_2021', 'drop_percent_2021'
                                     , 'g_2021', 'fppg_2021', 'fppg_2020'])

In [27]:
train_x_yds, test_x_yds, train_y_yds, test_y_yds = train_test_split(x_yds, y_yds, test_size =.2)

In [28]:
train_x_yds

,tgt/g_2020,rec/g_2020,yds/g_2020,td/g_2020,yac_r_2020,drop_percent_2020,g_2020
46,4.933333,3.266667,44.466667,0.133333,4.5,0.081,15.0
42,5.500000,3.437500,40.125000,0.312500,3.2,0.068,16.0
59,4.909091,2.818182,36.000000,0.090909,6.2,0.019,11.0
5,8.250000,6.250000,65.875000,0.625000,3.3,0.061,16.0
12,9.600000,5.866667,61.533333,0.466667,4.5,0.090,15.0
27,7.866667,4.400000,79.533333,0.266667,5.8,0.059,15.0
4,10.500000,7.142857,70.857143,0.571429,4.4,0.020,14.0
19,7.187500,4.750000,61.125000,0.562500,2.6,0.052,16.0
36,7.416667,4.916667,58.250000,0.500000,5.5,0.034,12.0
21,7.200000,4.933333,61.666667,0.933333,3.3,0.065,15.0


In [29]:
#train_x_yds = train_x_yds.drop(columns='age_y')
#test_x_yds = test_x_yds.drop(columns='age_y')
train_x_yds = train_x_yds.rename(columns={'tgt/g_2020':'tgt', 'rec/g_2020':'rec', 'yds/g_2020':'yds', 'td/g_2020':'td', 'yac_r_2020':'yac_r'
                   , 'drop_percent_2020':'drop_percent', 'g_2020':'g', 'fppg_2020':'fppg'})
test_x_yds = test_x_yds.rename(columns={'tgt/g_2020':'tgt', 'rec/g_2020':'rec', 'yds/g_2020':'yds', 'td/g_2020':'td', 'yac_r_2020':'yac_r'
                   , 'drop_percent_2020':'drop_percent', 'g_2020':'g', 'fppg_2020':'fppg'})

In [30]:
yds_model = XGBRegressor(learning_rate=.2, gamma = .5)
#yds_model = XGBRegressor()

In [31]:
yds_model.fit(train_x_yds, train_y_yds, verbose= False)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=0.5, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=0.2, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=None, n_jobs=None,
             num_parallel_tree=None, random_state=None, ...)

In [32]:
yds_prediction = yds_model.predict(test_x_yds)

In [33]:
error_yds = sum(abs(test_y_yds - yds_prediction))/len(test_y_yds)
error_yds

14.207638535052864

In [34]:
yds_prediction

array([87.791016, 55.844162, 58.993523, 62.974655, 92.22696 , 56.74038 ,
       56.651497, 33.834595, 58.294468, 70.664825, 34.92487 , 26.967306],
      dtype=float32)

In [35]:
#Off by an average of 15.3 yards per game

In [36]:
#Tgt predictions
#
#
#

In [37]:
y_rec = full_wr_data['rec/g_2021']
x_rec = full_wr_data.drop(columns =['player', 'tgt/g_2021', 'rec/g_2021', 'yds/g_2021'
                                     , 'td/g_2021', 'yac_r_2021', 'drop_percent_2021'
                                     , 'g_2021', 'fppg_2021', 'fppg_2020'])

In [38]:
train_x_rec, test_x_rec, train_y_rec, test_y_rec = train_test_split(x_rec, y_rec, test_size =.2)
#train_x_rec = train_x_rec.drop(columns='age_y')
#test_x_rec = test_x_rec.drop(columns='age_y')
train_x_rec = train_x_rec.rename(columns={ 'tgt/g_2020':'tgt', 'rec/g_2020':'rec', 'yds/g_2020':'yds', 'td/g_2020':'td', 'yac_r_2020':'yac_r'
                   , 'drop_percent_2020':'drop_percent', 'g_2020':'g', 'fppg_2020':'fppg'})
test_x_rec = test_x_rec.rename(columns={'tgt/g_2020':'tgt', 'rec/g_2020':'rec', 'yds/g_2020':'yds', 'td/g_2020':'td', 'yac_r_2020':'yac_r'
                   , 'drop_percent_2020':'drop_percent', 'g_2020':'g', 'fppg_2020':'fppg'})

In [39]:
rec_model = XGBRegressor(learning_rate = .25, gamma =.5)
#rec_model = XGBRegressor()

In [40]:
rec_model.fit(train_x_rec,train_y_rec, verbose = False)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=0.5, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=0.25, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=None, n_jobs=None,
             num_parallel_tree=None, random_state=None, ...)

In [41]:
rec_prediction = rec_model.predict(test_x_rec)

In [42]:
error_rec = sum(abs(test_y_rec - rec_prediction))/len(test_y_rec)
error_rec

0.854058523811292

In [43]:
#Recpetions per game off by an average of 1.2

In [44]:
#TD predictions
#
#
#

In [45]:
y_td = full_wr_data['td/g_2021']
x_td = full_wr_data.drop(columns =['player', 'tgt/g_2021', 'rec/g_2021', 'yds/g_2021'
                                     , 'td/g_2021', 'yac_r_2021', 'drop_percent_2021'
                                     , 'g_2021', 'fppg_2021', 'fppg_2020'])

In [46]:
train_x_td, test_x_td, train_y_td, test_y_td = train_test_split(x_td, y_td, test_size =.2)
#train_x_td = train_x_td.drop(columns='age_y')
#test_x_td = test_x_td.drop(columns='age_y')
train_x_td = train_x_td.rename(columns={ 'tgt/g_2020':'tgt', 'rec/g_2020':'rec', 'yds/g_2020':'yds', 'td/g_2020':'td', 'yac_r_2020':'yac_r'
                   , 'drop_percent_2020':'drop_percent', 'g_2020':'g', 'fppg_2020':'fppg'})
test_x_td = test_x_td.rename(columns={'tgt/g_2020':'tgt', 'rec/g_2020':'rec', 'yds/g_2020':'yds', 'td/g_2020':'td', 'yac_r_2020':'yac_r'
                   , 'drop_percent_2020':'drop_percent', 'g_2020':'g', 'fppg_2020':'fppg'})

In [47]:
td_model = XGBRegressor(learning_rate=.2)
#td_model = XGBRegressor()

In [48]:
td_model.fit(train_x_td, train_y_td, verbose= False)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=0.2, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=None, n_jobs=None,
             num_parallel_tree=None, random_state=None, ...)

In [49]:
td_prediction = td_model.predict(test_x_td)

In [50]:
error_td = sum(abs(test_y_td - td_prediction))/len(test_y_td)
error_td

0.12418378955219955

In [51]:
#TD prediction off by an average .186 TD

In [52]:
def calc_fppg_new(rec_g,yds_g,td_g):
    fppg = ((rec_g*1.0 + yds_g*.1 + td_g*6.0))
    return fppg

In [53]:
fppg_error = calc_fppg_new(error_rec, error_yds, error_td)
fppg_error

3.0199251146297756

# This section will attempt to predict stats for the 2023 NFL Season

In [74]:
season_data_2022 = nfl.import_seasonal_pfr('rec', [2022])

In [75]:
data_2022 = pd.DataFrame(season_data_2022,
                         columns = ['season', 'player', 
                                    'tm', 'pos','tgt','rec','yds','td','yac_r','drop_percent','g'])

In [76]:
wr_2022 = data_2022[data_2022['pos'] =='WR']
wr_2022 = wr_2022[wr_2022['tgt'] >= 50]
wr_2022 = wr_2022.drop(columns = ['season','tm'])

In [77]:
wr_2022 = wr_2022.reset_index()

In [78]:
wr_2022['fppg'] = np.empty(len(wr_2022), float).tolist()

In [79]:
for i in range(0,len(wr_2022)):
    wr_2022.loc[i,'fppg'] = calc_fppg(wr_2022.iloc[i]['rec'],
                                         wr_2022.iloc[i]['yds'],
                                         wr_2022.iloc[i]['td'], wr_2022.iloc[i]['g'])

In [80]:
#wr_2022 = wr_2022.rename(columns={'tgt':'tgt/g_2022', 'rec':'rec/g_2022', 'yds':'yds/g_2022', 'td':'td/g_2022', 'yac_r':'yac_r_2022'
 #                  , 'drop_percent':'drop_percent_2022', 'g':'g_2022', 'fppg':'fppg_2022'})

In [81]:
wr_2022['yds'] = wr_2022['yds']/wr_2022['g']
wr_2022['tgt'] = wr_2022['tgt']/wr_2022['g']
wr_2022['rec'] = wr_2022['rec']/wr_2022['g']
wr_2022['td'] = wr_2022['td']/wr_2022['g']

In [82]:
x_yds_2022 = wr_2022.drop(columns = ['player', 'index', 'fppg', 'pos'])

In [83]:
y_yds_2023 = yds_model.predict(x_yds_2022)

In [84]:
wr_2023 = wr_2022
wr_2023['yds'] = y_yds_2023
wr_2023 = wr_2023.drop(columns = ['index', 'yac_r', 'drop_percent', 'g'])

In [85]:
wr_2023

,player,pos,tgt,rec,yds,td,fppg
0,Justin Jefferson,WR,10.823529,7.529412,93.649734,0.470588,20.994118
1,Tyreek Hill,WR,10.000000,7.000000,57.582954,0.411765,19.529412
2,Stefon Diggs,WR,9.625000,6.750000,72.656075,0.687500,19.806250
3,CeeDee Lamb,WR,9.176471,6.294118,70.836250,0.529412,17.464706
4,Amon-Ra St Brown,WR,9.125000,6.625000,91.358360,0.375000,16.131250
...,...,...,...,...,...,...,...
80,Corey Davis,WR,4.923077,2.461538,27.502436,0.153846,7.507692
81,Nelson Agholor,WR,3.312500,1.937500,34.595882,0.125000,4.950000
82,DJ Chark,WR,4.727273,2.727273,29.384296,0.272727,8.927273
83,Trent Sherfield,WR,3.000000,1.764706,32.624146,0.117647,4.923529


In [86]:
x_rec_2022 = wr_2022.drop(columns = ['player', 'index', 'fppg', 'pos'])
y_rec_2023 = rec_model.predict(x_rec_2022)

In [87]:
wr_2023['rec'] = y_rec_2023

In [88]:
wr_2023

,player,pos,tgt,rec,yds,td,fppg
0,Justin Jefferson,WR,10.823529,6.390117,93.649734,0.470588,20.994118
1,Tyreek Hill,WR,10.000000,4.971262,57.582954,0.411765,19.529412
2,Stefon Diggs,WR,9.625000,5.457545,72.656075,0.687500,19.806250
3,CeeDee Lamb,WR,9.176471,5.457545,70.836250,0.529412,17.464706
4,Amon-Ra St Brown,WR,9.125000,5.010399,91.358360,0.375000,16.131250
...,...,...,...,...,...,...,...
80,Corey Davis,WR,4.923077,2.972551,27.502436,0.153846,7.507692
81,Nelson Agholor,WR,3.312500,2.863636,34.595882,0.125000,4.950000
82,DJ Chark,WR,4.727273,2.943978,29.384296,0.272727,8.927273
83,Trent Sherfield,WR,3.000000,2.863636,32.624146,0.117647,4.923529


In [89]:
x_td_2022 = wr_2022.drop(columns = ['player', 'index', 'fppg', 'pos'])
y_td_2023 = td_model.predict(x_td_2022)

In [90]:
wr_2023['td'] = y_td_2023

In [91]:
wr_2023 = wr_2023.drop(columns=['tgt'])

In [92]:
for i in range(0,len(wr_2023)):
    wr_2023.loc[i,'fppg'] = calc_fppg_new(wr_2023.iloc[i]['rec'],
                                         wr_2023.iloc[i]['yds'],
                                         wr_2023.iloc[i]['td'])

In [93]:
wr_2023.sort_values(by = 'fppg', ascending = False)

,player,pos,rec,yds,td,fppg
5,Chris Godwin,WR,6.682146,91.517181,0.752638,20.349691
11,JaMarr Chase,WR,7.020204,94.697540,0.592309,20.043811
22,Cooper Kupp,WR,6.813130,101.531273,0.405508,19.399303
6,Davante Adams,WR,5.934364,92.226959,0.688509,19.288112
4,Amon-Ra St Brown,WR,5.010399,91.358360,0.795838,18.921262
...,...,...,...,...,...,...
59,Noah Brown,WR,2.790069,26.388781,0.160108,6.389592
68,Michael Gallup,WR,2.790069,25.967993,0.110853,6.051984
47,Robert Woods,WR,2.615412,24.657131,0.129702,5.859335
57,Marvin Jones,WR,2.615412,24.700911,0.124422,5.832035
